# Amazon Bedrock AgentCore Policy - MCP Server Target Setup

## Overview

This notebook sets up the infrastructure for testing **Amazon Bedrock AgentCore Policy** with an **MCP Server Target** deployed to **AgentCore Runtime** with OAuth authentication.

### What This Notebook Creates

| Resource | Description |
|----------|-------------|
| AgentCore Runtime | Container hosting the MCP Server |
| MCP Server | FastMCP server with refund/claim tools |
| Cognito User Pool (Runtime) | OAuth authentication for Runtime |
| Cognito User Pool (Gateway) | JWT authentication for Gateway |
| AgentCore Gateway | MCP protocol endpoint with JWT Authorizer |
| Gateway Target | MCP Server connected via OAuth Credential Provider |

### Architecture

```
┌─────────────────┐             ┌───────────────────────┐             ┌─────────────────────┐
│   Gateway       │  JWT Token  │  AgentCore Gateway    │  MCP over   │  AgentCore Runtime  │
│   Cognito       │────────────>│  + JWT Authorizer     │  HTTP/OAuth │  (MCP Server)       │
│                 │             │                       │────────────>│                     │
└─────────────────┘             └───────────────────────┘             └─────────────────────┘
                                         │                                     │
                                         │                                     │ OAuth
                                         │                                     ▼
                                         │                            ┌─────────────────────┐
                                         │                            │  Runtime Cognito    │
                                         └───────OAuth Credential────>│  (Credential        │
                                             Provider                 │   Provider)         │
                                                                      └─────────────────────┘
```

> **Next Step**: After completing this notebook, run `02-Policy-Enforcement.ipynb` to test Cedar policies.

---

## Part 1: Environment Setup

### Prerequisites

- **bedrock-agentcore-starter-toolkit** installed
- **Docker** running (for container build)
- AWS credentials configured

### Setup Virtual Environment (Recommended)

```bash
cd ../00_setup
chmod +x create_uv_virtual_env.sh
./create_uv_virtual_env.sh AgentCorePolicyMCP
```

Then select the `AgentCorePolicyMCP` kernel in Jupyter.

In [ ]:
import json
import sys
import time
import os
from pathlib import Path
from urllib.parse import quote

import boto3
import requests

# Add parent directory to path for common imports
sys.path.insert(0, str(Path.cwd().parent))

from common.auth_utils import get_bearer_token, decode_token

print("✓ Libraries loaded")

In [ ]:
# Configuration
REGION = "us-east-1"
RUNTIME_NAME = "refund_mcp_server"
MCP_TARGET_NAME = "RefundMCPServerTarget"
SCRIPT_DIR = Path.cwd()

print(f"Region: {REGION}")
print(f"Runtime Name: {RUNTIME_NAME}")
print(f"Script Directory: {SCRIPT_DIR}")

---

## Part 2: Deploy MCP Server to AgentCore Runtime

This step deploys the MCP server using `deploy_mcp_runtime.py` which:
1. Creates Cognito User Pool for Runtime OAuth
2. Builds Docker container with MCP server
3. Pushes to ECR
4. Creates AgentCore Runtime with OAuth Authorizer

> ⏱️ This step takes approximately **5-10 minutes** for the first deployment.

In [ ]:
# Check if runtime is already deployed
runtime_config_file = SCRIPT_DIR / "runtime_config.json"
cognito_config_file = SCRIPT_DIR / "cognito_config.json"

if runtime_config_file.exists() and cognito_config_file.exists():
    print("✓ Runtime configuration already exists")
    print(f"  Config file: {runtime_config_file}")
    
    with open(runtime_config_file, "r") as f:
        runtime_config = json.load(f)
    print(f"  Runtime ID: {runtime_config.get('runtime_id')}")
    
    SKIP_DEPLOYMENT = True
else:
    print("Runtime not deployed yet. Will deploy in next cell.")
    SKIP_DEPLOYMENT = False

In [ ]:
# Deploy MCP Runtime (skip if already deployed)
if not SKIP_DEPLOYMENT:
    print("Deploying MCP Server to AgentCore Runtime...")
    print("This will take 5-10 minutes.")
    print("=" * 70)
    
    %run deploy_mcp_runtime.py
else:
    print("✓ Skipping deployment - runtime already exists")

---

## Part 3: Load Runtime Configuration

In [ ]:
# Load Runtime config
with open(runtime_config_file, "r") as f:
    RUNTIME_CONFIG = json.load(f)

# Load Cognito config (for Runtime OAuth)
with open(cognito_config_file, "r") as f:
    COGNITO_CONFIG = json.load(f)

RUNTIME_ID = RUNTIME_CONFIG["runtime_id"]
RUNTIME_ARN = RUNTIME_CONFIG["runtime_arn"]
MCP_URL = RUNTIME_CONFIG["mcp_url"]

print("✓ Configuration loaded")
print(f"  Runtime ID: {RUNTIME_ID}")
print(f"  Runtime ARN: {RUNTIME_ARN}")
print(f"  MCP URL: {MCP_URL[:80]}...")

---

## Part 4: Verify Runtime is Ready

In [ ]:
# Initialize AWS client
session = boto3.Session(region_name=REGION)
agentcore_client = session.client("bedrock-agentcore-control")
sts_client = session.client("sts")

ACCOUNT_ID = sts_client.get_caller_identity()["Account"]

print("✓ AWS clients initialized")
print(f"  Account ID: {ACCOUNT_ID}")

In [ ]:
# Check Runtime status
runtime_details = agentcore_client.get_agent_runtime(agentRuntimeId=RUNTIME_ID)
runtime_status = runtime_details.get("status")

print(f"Runtime Status: {runtime_status}")

if runtime_status == "READY":
    print("\n✓ Runtime is READY")
else:
    print(f"\n⚠️  Runtime status: {runtime_status}")
    print("   Please wait for runtime to be READY before continuing.")

---

## Part 5: Test Runtime Directly (Optional)

Test that the MCP server is responding correctly before connecting it to Gateway.

In [ ]:
# Get token for Runtime
token = get_bearer_token(
    token_endpoint=COGNITO_CONFIG["token_endpoint"],
    client_id=COGNITO_CONFIG["client_id"],
    client_secret=COGNITO_CONFIG["client_secret"],
    scope=COGNITO_CONFIG["scope"]
)

print(f"✓ Token received ({len(token)} chars)")

In [ ]:
# Test tools/list on Runtime directly
print("Testing MCP Server...")
print("=" * 70)

payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/list",
    "params": {}
}

response = requests.post(
    MCP_URL,
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}",
    },
    json=payload,
    timeout=30
)

print(f"Status: {response.status_code}")
result = response.json()

if "result" in result:
    tools = result["result"].get("tools", [])
    print(f"\n✓ MCP Server responding with {len(tools)} tools:")
    for tool in tools:
        print(f"  - {tool['name']}")
else:
    print(f"\n⚠️  Unexpected response:")
    print(json.dumps(result, indent=2))

---

## Part 6: Setup Gateway with MCP Target

Now we need to:
1. Create/reuse a Gateway with JWT Authorizer
2. Create OAuth Credential Provider for Runtime authentication
3. Add MCP Server as Gateway Target

> **Note**: This assumes you have an existing Gateway from `01-Lambda-Target` tutorial, or you can create a new one.

In [ ]:
# Check for existing Gateway config from 01-Lambda-Target
lambda_gateway_config = Path.cwd().parent / "01-Lambda-Target" / "gateway_config.json"

if lambda_gateway_config.exists():
    with open(lambda_gateway_config, "r") as f:
        GATEWAY_CONFIG = json.load(f)
    print("✓ Using existing Gateway from 01-Lambda-Target")
    print(f"  Gateway ID: {GATEWAY_CONFIG['gateway_id']}")
    USE_EXISTING_GATEWAY = True
else:
    print("⚠️  No existing Gateway found.")
    print("   Please run 01-Lambda-Target/01-Setup-Gateway-Lambda.ipynb first,")
    print("   or manually configure Gateway settings below.")
    USE_EXISTING_GATEWAY = False
    
    # Manual configuration (uncomment and fill in if needed)
    # GATEWAY_CONFIG = {
    #     "gateway_id": "YOUR_GATEWAY_ID",
    #     "gateway_url": "YOUR_GATEWAY_URL",
    #     "gateway_arn": "YOUR_GATEWAY_ARN",
    #     "client_info": {
    #         "client_id": "YOUR_CLIENT_ID",
    #         "client_secret": "YOUR_CLIENT_SECRET",
    #         "token_endpoint": "YOUR_TOKEN_ENDPOINT",
    #         "scope": "YOUR_SCOPE"
    #     }
    # }

In [ ]:
if USE_EXISTING_GATEWAY:
    GATEWAY_ID = GATEWAY_CONFIG["gateway_id"]
    GATEWAY_URL = GATEWAY_CONFIG["gateway_url"]
    GATEWAY_ARN = GATEWAY_CONFIG["gateway_arn"]
    
    # Save Gateway Cognito config for this tutorial
    gateway_cognito_config = {
        "client_id": GATEWAY_CONFIG["client_info"]["client_id"],
        "client_secret": GATEWAY_CONFIG["client_info"]["client_secret"],
        "token_endpoint": GATEWAY_CONFIG["client_info"]["token_endpoint"],
        "scope": GATEWAY_CONFIG["client_info"].get("scope", "")
    }
    
    gateway_cognito_file = SCRIPT_DIR / "gateway_cognito_config.json"
    with open(gateway_cognito_file, "w") as f:
        json.dump(gateway_cognito_config, f, indent=2)
    
    print("✓ Gateway configuration ready")
    print(f"  Gateway ID: {GATEWAY_ID}")
    print(f"  Gateway URL: {GATEWAY_URL}")

### Create OAuth Credential Provider

The Gateway needs an OAuth Credential Provider to authenticate with the MCP Runtime.

In [ ]:
# Create OAuth Credential Provider
CREDENTIAL_PROVIDER_NAME = "refund-mcp-server-identity"

# Check if credential provider already exists
try:
    existing_providers = agentcore_client.list_oauth2_credential_providers(
        tokenVaultName="default"
    ).get("credentialProviders", [])
    
    existing_provider = next(
        (p for p in existing_providers if p["name"] == CREDENTIAL_PROVIDER_NAME),
        None
    )
except Exception as e:
    existing_provider = None
    print(f"Note: {e}")

if existing_provider:
    CREDENTIAL_PROVIDER_ARN = existing_provider["credentialProviderArn"]
    print(f"✓ Using existing Credential Provider: {CREDENTIAL_PROVIDER_NAME}")
    print(f"  ARN: {CREDENTIAL_PROVIDER_ARN}")
else:
    print(f"Creating OAuth Credential Provider: {CREDENTIAL_PROVIDER_NAME}")
    
    try:
        response = agentcore_client.create_oauth2_credential_provider(
            tokenVaultName="default",
            name=CREDENTIAL_PROVIDER_NAME,
            credentialProviderVendor="CustomOAuth2",
            oauth2ProviderConfigInput={
                "customOAuth2ProviderConfig": {
                    "oauthDiscoveryUrl": COGNITO_CONFIG["discovery_url"]
                }
            },
            credentialLifetimeSeconds=3600
        )
        CREDENTIAL_PROVIDER_ARN = response["credentialProviderArn"]
        print(f"✓ Credential Provider created")
        print(f"  ARN: {CREDENTIAL_PROVIDER_ARN}")
    except Exception as e:
        print(f"⚠️  Error creating credential provider: {e}")
        CREDENTIAL_PROVIDER_ARN = None

In [ ]:
# Store client credentials in the Credential Provider
if CREDENTIAL_PROVIDER_ARN:
    try:
        agentcore_client.put_oauth2_credentials(
            tokenVaultName="default",
            credentialProviderName=CREDENTIAL_PROVIDER_NAME,
            oauth2CredentialRequest={
                "clientCredentialsRequest": {
                    "clientId": COGNITO_CONFIG["client_id"],
                    "clientSecret": COGNITO_CONFIG["client_secret"],
                    "scopes": [COGNITO_CONFIG["scope"]]
                }
            }
        )
        print("✓ Credentials stored in Credential Provider")
    except Exception as e:
        if "already exists" in str(e).lower() or "conflict" in str(e).lower():
            print("✓ Credentials already stored")
        else:
            print(f"⚠️  Error storing credentials: {e}")

### Create MCP Server Target on Gateway

In [ ]:
# Check for existing MCP target
existing_targets = agentcore_client.list_gateway_targets(
    gatewayIdentifier=GATEWAY_ID
).get("targets", [])

mcp_target = next(
    (t for t in existing_targets if t.get("name") == MCP_TARGET_NAME),
    None
)

if mcp_target:
    MCP_TARGET_ID = mcp_target["targetId"]
    print(f"✓ Using existing MCP Target: {MCP_TARGET_NAME}")
    print(f"  Target ID: {MCP_TARGET_ID}")
    print(f"  Status: {mcp_target.get('status')}")
else:
    print(f"Creating MCP Server Target: {MCP_TARGET_NAME}")
    
    # Create target with OAuth credential provider
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=GATEWAY_ID,
        name=MCP_TARGET_NAME,
        description="MCP Server for policy testing",
        targetConfiguration={
            "mcp": {
                "mcpServer": {
                    "url": quote(MCP_URL, safe=':/')
                }
            }
        },
        credentialProviderConfigurations=[
            {
                "credentialProviderType": "OAUTH2_CREDENTIAL_PROVIDER",
                "credentialProvider": {
                    "oauth2CredentialProviderArn": CREDENTIAL_PROVIDER_ARN
                }
            }
        ]
    )
    
    MCP_TARGET_ID = response["targetId"]
    print(f"✓ MCP Target created")
    print(f"  Target ID: {MCP_TARGET_ID}")

In [ ]:
# Wait for target to be ready
print("\nWaiting for target to be READY...")
max_wait = 120
start = time.time()

while time.time() - start < max_wait:
    target = agentcore_client.get_gateway_target(
        gatewayIdentifier=GATEWAY_ID,
        targetId=MCP_TARGET_ID
    )
    status = target.get("status")
    print(f"  Status: {status}")
    
    if status == "READY":
        print("\n✓ Target is READY")
        break
    elif status in ["FAILED", "CREATE_FAILED"]:
        print(f"\n✗ Target failed: {target.get('statusReason', 'Unknown')}")
        break
    
    time.sleep(10)

In [ ]:
# Synchronize target to discover tools
print("Synchronizing target to discover tools...")

try:
    agentcore_client.synchronize_gateway_targets(
        gatewayIdentifier=GATEWAY_ID,
        targetId=MCP_TARGET_ID
    )
    print("✓ Synchronization initiated")
    print("  Waiting for sync to complete...")
    time.sleep(15)
except Exception as e:
    print(f"⚠️  Sync error: {e}")

---

## Part 7: Save Configuration

In [ ]:
# Update runtime_config.json with Gateway and Target info
RUNTIME_CONFIG["gateway_id"] = GATEWAY_ID
RUNTIME_CONFIG["gateway_url"] = GATEWAY_URL
RUNTIME_CONFIG["gateway_arn"] = GATEWAY_ARN
RUNTIME_CONFIG["target_id"] = MCP_TARGET_ID
RUNTIME_CONFIG["credential_provider_arn"] = CREDENTIAL_PROVIDER_ARN

with open(runtime_config_file, "w") as f:
    json.dump(RUNTIME_CONFIG, f, indent=2)

print("✓ Configuration saved to runtime_config.json")

---

## Part 8: Verify Setup

Test that we can invoke the MCP tool through the Gateway.

In [ ]:
# Get Gateway token
gateway_cognito_file = SCRIPT_DIR / "gateway_cognito_config.json"
with open(gateway_cognito_file, "r") as f:
    gateway_cognito = json.load(f)

gateway_token = get_bearer_token(
    token_endpoint=gateway_cognito["token_endpoint"],
    client_id=gateway_cognito["client_id"],
    client_secret=gateway_cognito["client_secret"],
    scope=gateway_cognito.get("scope", "")
)

print(f"✓ Gateway token received ({len(gateway_token)} chars)")

In [ ]:
# Test tool call through Gateway
print("Testing MCP tool through Gateway...")
print("=" * 70)

TOOL_NAME = f"{MCP_TARGET_NAME}___approve_claim"

payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/call",
    "params": {
        "name": TOOL_NAME,
        "arguments": {
            "claim_id": "TEST-001",
            "amount": 100,
            "risk_level": "low"
        }
    }
}

response = requests.post(
    GATEWAY_URL,
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {gateway_token}",
        "Accept": "application/json"
    },
    json=payload,
    timeout=30
)

print(f"Status: {response.status_code}")
result = response.json()
print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

if "result" in result and not result["result"].get("isError"):
    print("\n✓ Setup verification successful!")
    print("  Gateway can invoke MCP tools.")
elif "error" in result:
    error_msg = result["error"].get("message", "")
    if "not allowed" in error_msg.lower() or "denied" in error_msg.lower():
        print("\n⚠️  Request denied by policy.")
        print("   This is expected if policies are attached.")
        print("   Run 02-Policy-Enforcement.ipynb to configure policies.")
    else:
        print(f"\n⚠️  Error: {error_msg}")

---

## Summary

This notebook has set up:

✅ AgentCore Runtime with MCP Server  
✅ Cognito User Pool for Runtime OAuth  
✅ OAuth Credential Provider  
✅ MCP Server Target on Gateway  
✅ Tool discovery via SynchronizeGatewayTargets  

### Configuration Files Created

| File | Contents |
|------|----------|
| `runtime_config.json` | Runtime ID, ARN, Gateway info |
| `cognito_config.json` | Runtime Cognito credentials |
| `gateway_cognito_config.json` | Gateway Cognito credentials |

### Next Step

Run **`02-Policy-Enforcement.ipynb`** to:
- Create Policy Engine and attach to Gateway
- Test Cedar policies for fine-grained access control
- Learn string equality, pattern matching, OR conditions, and negation patterns